In [1]:
# This is in case you update modules while working
%load_ext autoreload 
%autoreload 2

# Bad Crawler

Experiment to test traditional HTML crawling. Inefficient and it probably overloads the library website.

In [83]:
from urllib.request import urlretrieve
import requests
from bs4 import BeautifulSoup
import time
import shutil, os.path

In [119]:
pages = []
for i in range(1, 6):
    url = ('https://repository.tudelft.nl/islandora/search?page=' + str(i) + 
    '&collection=education&f%5B0%5D=mods_genre_s%3A%22master%5C%20thesis%22')
    pages.append(url)

In [123]:
i = 0
j = 0
for item in pages:
    page = requests.get(item)
    soup = BeautifulSoup(page.text, 'html.parser')
    thesis_list = soup.find(class_='islandora-basic-collection-list')
    thesis_list_items = thesis_list.find_all('a')
    print(item)
    for thesis_name in thesis_list_items:
        url = thesis_name.get('href')
        uuid = url[25:-21]
        download = 'https://repository.tudelft.nl/islandora/object/uuid:' + uuid + '/datastream/OBJ/download'     
        name = 'data/tudelft_repo/TUD_' + uuid + '.pdf' 
        if os.path.exists(name):
            print('File exists:', name)
            i = i + 1
            continue
        r = requests.get(download, stream=True)
        if r.status_code == 200:
            with open(name, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
            print('Downloaded:', name)
            j = j + 1
        if r.status_code != 200:
            ('Download failed, status', r.status_code, 'link', download)

        time.sleep(1)
        
print(i, 'files already existed,', j, 'files downloaded')

https://repository.tudelft.nl/islandora/search?page=1&collection=education&f%5B0%5D=mods_genre_s%3A%22master%5C%20thesis%22
File exists: data/tudelft_repo/TUD_1575733a-8813-4c5d-b916-0eb23ac477f9.pdf
File exists: data/tudelft_repo/TUD_05957f39-d946-4956-be68-52f5675e100d.pdf
File exists: data/tudelft_repo/TUD_e675fadc-fd8d-4ced-b59e-45b6faba6fdd.pdf
File exists: data/tudelft_repo/TUD_c9492ef8-9254-4ba4-83b3-8b0ad46149c5.pdf
File exists: data/tudelft_repo/TUD_6e274095-9920-4d20-9e11-d5b76363e709.pdf
File exists: data/tudelft_repo/TUD_b2308402-21ed-41a2-92b7-7edd7175dca7.pdf
File exists: data/tudelft_repo/TUD_c46eca4b-eaf5-461a-9be2-dd6b3b1b44e0.pdf
File exists: data/tudelft_repo/TUD_ca8da1e3-94c0-438b-9281-ee4a76cb7129.pdf
File exists: data/tudelft_repo/TUD_15227443-568e-4bd8-af88-a165c4776f11.pdf
File exists: data/tudelft_repo/TUD_cff45740-9100-4a10-9d49-41da874146d5.pdf
https://repository.tudelft.nl/islandora/search?page=2&collection=education&f%5B0%5D=mods_genre_s%3A%22master%5C%20th

# OAI Extraction

Obrain metadata from the OAI-PMH site of the Library (Open Archives Initiative Protocol for Metadata Harvesting).

In [2]:
from urllib.request import urlretrieve
from bs4 import BeautifulSoup 
from sickle import Sickle
import shutil, os.path, pickle, time, requests
from collections import Counter

In [3]:
sickle = Sickle('http://oai.tudelft.nl/ir')

In [100]:
filter_types = ['master thesis', 'doctoral thesis']
items = {}
records = sickle.ListRecords( **{'metadataPrefix': 'oai_dc', 'ignore_deleted': 'True'})
for item in records:
#     item = records.next()
    uuid = item.metadata['identifier'][0][32:]
    try:
        if item.metadata['type'][0] in filter_types and item.metadata['description']:
            items[uuid] = item.metadata
    except KeyError:
        continue
    
print(len(items))

24539


In [53]:
fac = []
for uuid in items:
    try:
        fac.append(items[uuid]['description'][1])
    except IndexError:
        continue

In [57]:
fac = []
for uuid in items:
    if len(items[uuid]['description']) > 1:
        tmp = items[uuid]['description'][1]
        if 'computer' in tmp.lower():
            fac.append(items[uuid]['description'][1])

In [58]:
Counter(fac)

Counter({'Computer Engineering': 7,
         'Computer Science': 3,
         'Computer Science, Embedded Software': 1,
         'Computer and Information Science - Software Engineering': 1,
         'Electrical Engineering, Mathematics and Computer Science': 18,
         'Master of Science Computer Science': 1,
         'Microelectronics & Computer Engineering': 4})

In [345]:
x = 0
types = []
items = {}

records = sickle.ListRecords( **{'metadataPrefix': 'oai_dc', 'ignore_deleted': 'True'})
for r in records:
    types.append(r.metadata['type'][0])
    uuid = ''
    uuid = r.metadata['identifier'][0][32:]
    items[uuid] = r.metadata
    x = x + 1

print('items', len(items), 'types', len(types), 'x', x)
Counter(types) 

items 57771 types 62105 x 62105


Counter({'Text': 1,
         'abstract': 155,
         'bachelor thesis': 1128,
         'book': 1926,
         'book chapter': 525,
         'conference paper': 10767,
         'contribution to periodical': 29,
         'doctoral thesis': 8331,
         'journal article': 14483,
         'lecture notes': 489,
         'master thesis': 16201,
         'other': 77,
         'patent': 544,
         'public lecture': 1228,
         'report': 5797,
         'review': 46,
         'student report': 300,
         'working paper': 78})

In [ ]:
with open('tud_metadata.pickle', 'wb') as handle:
    pickle.dump(items, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [403]:
with open('tud_metadata.pickle', 'rb') as handle:
    items = pickle.load(handle)  

In [ ]:
failed = []
for uuid in items.keys():
    download = 'https://repository.tudelft.nl/islandora/object/uuid:' + uuid + '/datastream/OBJ/download'     
    name = 'data/tudelft_repo/TUD_' + uuid + '.pdf' 
    if os.path.exists(name):
        print('File exists:', name)
        print('.', end = '')
        continue
    r = requests.get(download, stream=True)
    if r.status_code == 200:
        with open(name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
        print('Downloaded:', name)
    if r.status_code != 200:
        print('Download failed, status', r.status_code, 'link', download)
        failed.append(uuid)
    time.sleep(3)

In [ ]:
for uuid in items.keys():
    download = 'https://repository.tudelft.nl/islandora/object/uuid:' + uuid + '/datastream/OBJ/download'
    name = working_dir + 'TUD_' + uuid + '.pdf'
    if os.path.exists(name):
        print('File exists:', name)
        print('.', end='')
        continue
    r = requests.get(download, stream=True)
    if r.status_code == 200 and items[uuid]['type'][0] in filter_types:
        with open(name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
        print('Downloaded:', name)
        x += 1
    if r.status_code != 200:
        print('Download failed, status', r.status_code, 'link', download)
        failed.append(uuid)
    time.sleep(1)
    if x == 20:
        break

In [59]:
downloaded = {}
for dirpath, dirs, files in os.walk('data/tudelft_repo/'):
    for file in files: 
        path = os.path.join(dirpath, file)
        uuid = path.split('_')[-1].split('.')[0]
        downloaded[uuid] = items[uuid]

KeyError: '200f6974-3d2c-4383-87bd-1760fad834d8'

In [66]:
uuids = items.keys()

In [70]:
items[list(uuids)[0]]

{'contributor': ['Hermans, L.M. (mentor)'],
 'creator': ['Reijtenbagh, R. (author)'],
 'date': ['2010-10-05'],
 'description': ['Rainwater harvesting is the primary source of fresh water on the majority of isolated and volcanic islands in the Eastern Caribbean island chain that lack reliable sources fresh water. Recent research studies have determined that the microbiological quality of rainwater in cistern systems is generally unstable and can lead to serious health implications when used for human consumption. Another related issue is that rainwater shortages occur more frequently due to fluctuating rainfall patterns and increasing rainwater consumption rates. In this paper minimum design requirements are determined, using data from the islands of Saba and St Eustatius, for the construction of new domestic rainwater systems on small volcanic islands in the Eastern Caribbean. These minimum design requirements can be used to prevent potential water quality and quantity issues. One of t

# Delete Large Files

For easier processing. There are some very heavy pdfs that may not be necessary.

In [1]:
import os

target_size = 100 * 1048514
large = []

for dirpath, dirs, files in os.walk('data/tudelft_repo/'):
    for file in files: 
        path = os.path.join(dirpath, file)
        if os.stat(path).st_size > target_size:
            os.remove(path)
            large.append(file)

In [271]:
with open('large_deleted.pickle', 'wb') as handle:
    pickle.dump(large, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Collection in Mongo

In [7]:
import pymongo
import elasticsearch
from elasticsearch import helpers
import nltk
import config as cfg

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
client = pymongo.MongoClient('localhost:' + str(cfg.mongoDB_Port))
publications_collection = client.TU_Delft_Library.publications
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}],
                                 timeout=30, max_retries=10, retry_on_timeout=True)


In [13]:
def extract_metadata(documents):
    list_of_docs = []
    for i, r in enumerate(documents):
        extracted = {
                "_id": "",
                "title": "",
                "publication": "",
                "year": "",
                "content": "",
                "abstract": "",
                "authors": [],
                "references": []}
        extracted['_id'] = r['_id']
        extracted['title'] = r['title']
        extracted['publication'] = r['faculty']
        extracted['year'] = r['year']
        try:
            extracted['content'] = r['content']['fulltext']
        except KeyError:
            pass
        try:
            extracted['abstract'] = r['content']['abstract']
        except KeyError:
            extracted['abstract'] = ''
        extracted['authors'] = r['authors']
        try:
            extracted['references'] = r['content']['references']
        except KeyError:
            extracted['references'] = ''
        list_of_docs.append(extracted)
    return list_of_docs

In [14]:
extracted_publications = []
query = {}
results = publications_collection.find(query)
extracted_publications.append(extract_metadata(results))

In [22]:
len(extracted_publications[0])

955

In [21]:
extracted_publications[0][0].keys()

dict_keys(['content', 'authors', 'title', 'abstract', '_id', 'year', 'references', 'publication'])

In [120]:
for doc in extracted_publications[0]:
    _id = doc['_id']
    uuid = _id[4:]
    mentors = []
    promotors = []
    try:
        for c in items[uuid]['contributor']:
            if 'mentor' in c:
                mentors.append(c)
                publications_collection.update_one({'_id': _id}, {'$set' : {'mentors' : mentors}})
            if 'promotor' in c:
                promotors.append(c)
                publications_collection.update_one({'_id': _id}, {'$set' : {'promotors' : promotors}})
    except KeyError:
        continue

TUD_862e5b58-b9d1-462a-90b0-6f94a054632e
TUD_960eb3cd-080d-4423-b9cb-f82417e9e13c
TUD_c35fa6e0-e632-4f17-bf84-0f1cc8f98c0c
TUD_f116361b-8ff7-4995-a8e5-e5354f2902b9
TUD_46081d76-17fc-41f6-bdc4-fbdc6d7bb939
TUD_ed609f8d-6cc5-489f-aa45-669b797f677a
TUD_f514548b-63c6-4015-a824-c86c9a63477c
TUD_beca2029-d274-4d14-9ea0-a59490019bc1
TUD_fc4aa0e6-1d20-41e9-a660-237a6873bb3c
TUD_84ff0f8e-ec0a-4b77-8fa9-9df7eaef6b60
TUD_9ced4cb2-9809-4b58-af25-34e458e8ea23
TUD_b0c2a859-4351-4b6a-b66b-a7bbfed76ecb
TUD_280fe4d8-d1de-4a46-ad99-be8129d85dab
TUD_78bf003b-e784-40ca-888c-48e0246b3883
TUD_ba867fef-5421-4692-b955-6c2c468653cc
TUD_b6ea3435-825a-4306-a4d4-4edee26f233c
TUD_8150156e-e633-4319-8685-ccac7b083434
TUD_c061a03a-84a5-4ac3-ba22-c5b315f08edd
TUD_32789615-a48a-4b0f-ba27-d11992a201d1
TUD_187e5e62-db64-4f62-8b08-c5dddfc174a3
TUD_22662036-5abb-4a5a-9f1a-a4a9a68e0997
TUD_d8f95cd0-f243-4989-89f3-a8a262b1bfd6
TUD_8bd86467-7565-481c-beec-4d76423f1bff
TUD_167d7879-7f61-4c5e-9390-7d1435e9eda1
TUD_a69af610-68f

TUD_5c01eb69-0d5b-4016-8b55-b450e87c24e2
TUD_e7ed020a-ee53-4010-a14c-4db3ae31a01b
TUD_a22c55d7-4c69-44c9-bc9f-03652b4ddfdf
TUD_adda5ce3-0931-4282-8086-fb0cb4fc3d5d
TUD_740995b0-2576-42ea-aa49-940c6e10a7ba
TUD_bbed8ce5-36a8-4773-a461-33f19c5f774d
TUD_f43422a6-ef31-4f61-ae7a-f71d3b3c645b
TUD_9328b51a-7eb5-4206-b463-48d99b86aafd
TUD_c0c07509-513f-4aa1-a07c-78f5c9ff5684
TUD_b806e4df-0982-4b2e-9e63-7299ab3a8bff
TUD_3758d2cb-d0af-45d1-9f14-93e1c86f60ed
TUD_204b09ee-971b-4035-a4b1-616ee50afbba
TUD_74af4924-f262-4136-8f44-29c6be14534e
TUD_8ef36410-06c2-4372-a9fe-e0e45b264a73
TUD_40148cf1-8b85-49df-9f5a-55ac257a04f7
TUD_81f376a3-2c9a-4d24-b547-6b384dc0aef6
TUD_493487ad-ac1e-4ad5-9264-48be7ebc2f74
TUD_68ef1d61-f3c3-43e8-bacd-d2e97bd0baa1
TUD_4fd01d95-6bf6-4864-8662-f7adf837e2f5
TUD_f17092aa-4d78-4ef3-a80c-365825e810f1
TUD_117aa738-99ea-4357-b880-769b58c84d97
TUD_c3faec41-007f-4b2c-acbe-eb2eb89efd32
TUD_d229c59f-c03f-4907-b9e8-67b83b619ac3
TUD_29251f71-81ea-475c-8a3f-5795a6d1b341
TUD_2eb0995d-2c5

TUD_f444e6d3-cfa1-47dc-9fb4-9ca6978b4249
TUD_bc53cc52-8507-43a0-9325-d4304067f677
TUD_486154d6-b535-4a2a-b394-c2f7f3f43ac4
TUD_b913c0ee-fa19-44a6-a7ef-41fecde7125c
TUD_5bb2bb3c-c545-4d7f-9502-186ac0f5956e
TUD_6a2cad07-86c3-4a16-9d04-1224ef4d900f
TUD_1a95dc0b-64f1-42f1-9971-1f8bcb146b1f
TUD_682a56ed-8e21-4b70-af11-0e8e9e298fa2
TUD_218b896a-af74-4637-a0e7-fcd4e4743fed
TUD_3e8a66e5-80f4-466d-b6bb-961f9896b800
TUD_dd58900a-a201-474b-b02a-4feab83b25d1
TUD_60fee9f1-7f5a-4915-8f09-2a3bcdbdd55c
TUD_39b1653b-cde7-419b-bcd0-8549b6e34db5
TUD_4d4e1516-3c86-4b78-8c13-bf89953af029
TUD_ff57aedb-20f8-42cb-bc6f-513b8c8e94c5
TUD_98015359-f99c-42a0-bb63-4e99c975decc
TUD_a1689901-8ebe-4d2a-bab9-9e9569e3d83c
TUD_6980f3d8-362c-412b-a560-565fd14d59c4
TUD_29c201ec-0813-45b7-8d52-53371868150c
TUD_5050b8ae-184b-40c6-9640-c4ba7fb875c6
TUD_076ec1f6-d6c6-4266-afc5-dd71e24d65b2
TUD_0eb48447-21ff-4202-85c0-25017c9fea6a
TUD_3f184afb-241d-4d54-827d-75fcd59211a5
TUD_e9b513c8-751b-45a7-9d99-a51177c918a2
TUD_70ad3815-d28

TUD_566b7b54-b40b-45fa-85c2-debaaa8098d4
TUD_4439ccef-1079-43ec-96b5-d143eda4e23d
TUD_c107f01a-3fb7-4dad-99b9-240cbdae2b90
TUD_95a72bc6-2246-46ec-aa3d-9c222b11ceb0
TUD_b242650e-9420-4cc2-b851-9bac0af62b0d
TUD_eab4f47a-17c6-40c7-b55a-da09a5813cf8
TUD_78eed80f-2d9e-4c9a-b806-40b8adb9ccda
TUD_a5157424-63cf-4770-9e07-20c35971a84c
TUD_7e5ccc35-774e-41d6-b12a-a502b1960256
TUD_7bcddd81-b8de-4bcc-a069-7080bef1bdf6
TUD_8fb5b6f6-93a6-4aed-a446-013125541ff9
TUD_136cd4d6-396a-4520-98bf-d9dc9782f807
TUD_bd7ab4e8-602d-4b24-a9ce-63313e47e6b0
TUD_be7f6eee-e1e7-40d4-b2f8-5922139153b2
TUD_20170a8c-7e1e-434f-b5d6-9ba933e2ab6e
TUD_a3bea0c0-fc6d-47a1-9b94-bf39df4aff8f
TUD_4f9e5af1-bbda-4685-9045-2014171fb928
TUD_fdaf7d6a-1d45-41a5-8157-15586fa3480f
TUD_93fba141-d466-468d-96d5-6991e5b658cf
TUD_ae14fcf5-707e-4666-8b34-f3a91fd25f78
TUD_db5f3d18-ce49-496a-8328-ec9203136949
TUD_43902b9c-ed6b-4743-b00b-d8f803b111e0
TUD_3a4b2767-6172-4eeb-a6b4-1ddae7b531c5
TUD_01bfcad0-9a56-454b-a30e-dc2fd3394f12
TUD_fd98efb8-822

In [23]:
for doc in extracted_publications[0]:
    _id = doc['_id']
    uuid = _id[4:]
    publications_collection.update_one({'_id': _id}, {'$set' : {'journal' : 'tudelft'}})

In [121]:
items[uuid]['type'][0]

'master thesis'

# Indexing and ES

In [44]:
res = es.search(index = "ir_tud", body = {"query": {"match": {"title" : "machine"}}}, size = 20)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])

Got 5 Hits:
TUD_20170a8c-7e1e-434f-b5d6-9ba933e2ab6e A machine learning approach for optimisation in railway planning
TUD_d4d2230a-a034-43fa-bd29-0eb6b954410b Scalable machine learning algorithms on a big data infrastructure
TUD_25aea2bb-cd25-499f-9fe6-8de69462e49d Modelling and optimization of a permanent-magnet machine in a flywheel
TUD_33c6e093-a129-47ec-a663-3ce808343a93 Design of a high speed induction starter/generator for aerospace applications: Comparison with PM machine
TUD_98a8d01e-6259-4fed-8704-5af0615304d2 Development of the Machine Learning-Based Approach to Access Occupancy Information Through Indoor CO2 Data: for Demand-Driven Building Heating Energy Management


In [160]:
import pymongo
import elasticsearch
from elasticsearch import helpers
import nltk
import sys
import os
import config as cfg

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
client = pymongo.MongoClient('localhost:' + str(cfg.mongoDB_Port))
publications_collection = client.TU_Delft_Library.publications
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}],
                                 timeout=30, max_retries=10, retry_on_timeout=True)
es.cluster.health(wait_for_status='yellow', request_timeout=1)

def extract_metadata(documents):
    list_of_docs = []
    for i, r in enumerate(documents):
        extracted = {
                "_id": "",
                "title": "",
                "publication": "",
                "year": "",
                "content": "",
                "abstract": "",
                "authors": [],
                "references": []}
        extracted['_id'] = r['_id']
        extracted['title'] = r['title']
        extracted['faculty'] = r['faculty']
        extracted['year'] = r['year']
        extracted['type'] = r['type']
        extracted['authors'] = r['authors']
        extracted['content'] = r['content']['fulltext']

        try:
            extracted['mentors'] = r['mentors']
        except KeyError:
            pass
        
        try:
            extracted['promotors'] = r['promotors']
        except KeyError:
            pass  
        
        try:
            extracted['abstract'] = r['content']['abstract']
        except KeyError:
            extracted['abstract'] = ''
            
        try:
            extracted['references'] = r['content']['references']
        except KeyError:
            extracted['references'] = ''
        
        list_of_docs.append(extracted)
    return list_of_docs


In [166]:
extracted_publications = []
query = {}
results = publications_collection.find(query)

In [167]:
for r in results:
    print(r['title'])

Hybrid Interconnect Design for Heterogeneous Hardware Accelerators
Modeling and Inversion of Scattered Surface waves
Hardware Acceleration of Bioinformatics Sequence Alignment Applications
Low-Power Receive-Electronics for a Miniature 3D Ultrasound Probe
Removing roadblocks for mobile phone sensing
Analyzing, Designing & Implementing a Secure, Centralized, Generic, Extensible, Input-Sanitizing, Intuitive Configuration Management Station
Efficient numerical methods for the instationary solution of laminar reacting gas flow problems
60 GHz Beamforming Transmitter Design for Pulse Doppler Radar
Task Scheduling for Adaptive Reconfigurable VLIW Multicore Processors
Implementing and evaluating a simplified transistor model for timing analysis of integrated circuits
Food Benefit-Risk Assessment with Bayesian Belief Networks and Multivariable Exposure-Response
Reducing Free Riding in Peer-to-Peer Systems with Supervised Teaming
?-VEX: A reconfigurable and extensible VLIW processor
Modelling co

Hierarchically semi-separable representation and its applications
Random Subspace Method for One-Class Classifiers
DotFETs: MOSFETs strained by a Single SiGE dot in a Low-Temperature ELA Technology
Advanced Retrieval Models for Web Image Search
First approach for a decentralized resource allocation in ad hoc grids
K-loops: Loop Transformations for Reconfigurable Architectures
Optimization of on-site PD measurements and evaluation of diagnostic parameters for assessing condition of distribution cable system
Ferroelectric Materials and Artificial Dielectric Layer Structures for Microwave Integrated Circuit Technologies
Modeling and optimization of Nb tuning circuits for Nb/AlOx/Nb SIS mixers in the 600-720 GHz range
A Universal Fault Injector for Structural Simulation Environments
Highly Linear LNA Design for Base Station Applications
Integration of existing optimisation techniques with the DWARV C-to-VHDL compiler
Enterprise Expertise Characterization
Exploiting the Reconfigurability of

Algorithms for Simple Temporal Reasoning
Flash memory device: Electrical modeling and simulation
90 nm VLSI Design of an 8-bits Microcontroller
Comparing Feature Sets and Classifiers for Sentiment Analysis of Opinionated Free Text
What Does Passive Learning Bring To Adyen?
Performance Validation of Networks on Chip
SmArT solving: Tools and techniques for satisfiability solvers
Application of Monte Carlo Simulation for risk-based decision making in MV distribution networks
Kafkalytics: Anti-pattern detection in a Kafka message bus
Ecological approach to develop a shared mental model for collaboration at a distance during a crisis situation
PPU: A Protocol Parsing Unit in Hardware
Testing of modern semiconductor memory structures
Performance Analysis of WebRTC-based Video Conferencing
Shai: Scenario authoring for simulation games
Evaluation Framework for Task Scheduling Algorithms in Distributed Reconfigurable Systems
The impact of event-triggered control on the energy consumption of a l

Multi Bit-Rate Video on Demand for P2P networks
Research, design, simulation and implementation of an automatic flight control system for a real-time flight simulator
Superlinearly Convergent Unstructured Staggered Schemes for Compressible and Incompressible Flows
Quantitative analysis of trade-offs and model input sensitivities in public health
Analysis of infinite dimensional diffusions
Designing the Real-Valued Gene-pool Optimal Mixing Evolutionary Algorithm and Applying it to Substantially Improve the Efficiency of Multi-Objective Deformable Image Registration
Validating perceptual objective listening quality assessment methods on the tonal language Igbo
Suitability of tile-based rendering for low-power 3d graphics accelerators
Sampling time error calibration in Time-interleaved ADCs
Credits in BitTorrent: designing prospecting and investment functions
A Semantic Web based approach to expertise finding at KPMG
Run-time Adaptable VLIW Processors: Resources, Performance, Power Consum

User Modeling and Personalization in the Microblogging Sphere
Transparency in actions on requirements
Evaluation of innovative land tools in sub-Saharan Africa: Three cases from a peri-urban context
The world according to MARP
A Variational Formulation for Thin Membranes
Perspectives on Cyber Security: Managerial perspectives on cyber security and the role of end user awareness
Personalized Microblog Search on Twitter
Architecture-Driven Integration of Modeling Languages for the Design of Software-Intensive Systems
Computational Architecture: Focusing on perception and functionality aspects of urban intervention
A mathematical analysis of a belt system with a low and time-varying velocity
Performance Evaluation of Polybenzimidazole for Potential Aerospace Applications
Modelling and optimization of a permanent-magnet machine in a flywheel
An Art Museum Hub for Den Haag
Ultrasonic Plastic Welding of Carbon Fiber Reinforced Polyamide 6 to Aluminium and Steel
Taming Technology: The narrati

In [54]:
from elasticsearch import Elasticsearch
import pandas as pd
from nltk.corpus import wordnet
import operator, requests
from pymongo import MongoClient
from xml.etree import ElementTree
import operator

client = MongoClient('localhost:4321')
db = client.TU_Delft_Library

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [39]:
query_input = 'Lofi'

In [40]:
# _query_author = {
#     'query': {
#         'match_phrase': {
#             'authors': query_input        }
#     }
# }

_query_author = {
                  "query": {
                    "bool": {
                      "should": [
                        { "match": { "authors":  query_input }},
                        { "match": { "supervisors": query_input   }}
                      ]
                    }
                  }
                }

# _query_author = {"query":
#     {"match": {
#         "authors": {
#             "query": string,
#             "operator": "and"
#         }
#     }
#     }
# }

_search_author = es.search(index="ir_tud", doc_type="publications", body=_query_author)
for doc in _search_author['hits']['hits']:
#     title_list.append(doc['_source']['title'])
#     journal_list.append(doc['_source']['journal'])
#     year_list.append(doc['_source']['year'])
#     authors_list.append(doc['_source']['authors'])
#     id_list.append(doc['_id'])
    print(doc['_source']['authors'])
    print(doc['_source']['supervisors'])
    print(doc['_source']['title'])

['Valle Torre, Manuel (author)']
['Lofi, Christoph (mentor)']
Shared Perspectives on Perceptual Features for Query by Example in E-Commerce Systems
['Zegers, Jan (author)']
['Lofi, Christoph (mentor)']
Modelling and Mining of Data mining Workflows in Data Science Scripts


In [48]:
_string = 'machine learning'

_query_all = {
    'query': {
        "query_string": {
            "default_field": "content",
            "query": _string
        }
    }
}

_search = es.search(index="ir_tud", doc_type="publications", body=_query_all, size = 20)

In [51]:
id_list = []
for doc in _search['hits']['hits']: 
#     print(doc['_source']['title'])
    id_list.append(doc['_id'])

In [52]:
def popular_upcoming_entities(paper_id_list):
    terms_in_results = []
    terms_labels = {}
    entity_occurrences = {}
    upcoming_occurrences = {}

    for paper in paper_id_list:
        _query_terms = {
            "query": {
                "bool": {
                    "must": [{"match": {'paper_id': paper}}],
                    "should": [{"match": {"annotator": 'method'}},
                               {"match": {"annotator": 'dataset'}}]
                }
            }
        }

        _query_terms = es.search(index="entities_tud", doc_type="entities", body=_query_terms)

        for hit in _query_terms['hits']['hits']:
            entity = hit['_source']['lower']
            terms_in_results.append(entity)
            terms_labels[entity] = hit

    terms_in_results = list(set(terms_in_results))

    for entity in terms_in_results:
        _query_occurrences = {
            'query': {
                'match_phrase': {
                    'content': entity
                }
            }
        }

        _query_occurrences = es.search(index="ir_tud", doc_type="publications", body=_query_occurrences)
        years = []
        entity_occurrences[entity] = _query_occurrences['hits']['total']

        for doc in _query_occurrences['hits']['hits']:
            years.append(doc['_source']['year'])
        if years:
            if int(min(years)) > 2010:
                upcoming_occurrences[entity] = _query_occurrences['hits']['total']

    sorted_occurrences = sorted(entity_occurrences.items(), key=operator.itemgetter(1), reverse=True)
    sorted_upcoming = sorted(upcoming_occurrences.items(), key=operator.itemgetter(1), reverse=True)

    triples = []
    for pair in sorted_occurrences:
        amount = pair[1]
        term = pair[0]

        if terms_labels[term]['_source']['annotator'] in ['method', 'dataset']:
            actual_label = terms_labels[term]['_source']['annotator']
        elif terms_labels[term]['_source']['annotator'] in ['other', 'noise', 'software']:
            continue
        elif terms_labels[term]['_source']['mt_similarity'] > terms_labels[term]['_source']['ds_similarity']:
            actual_label = 'method'
        else:
            actual_label = 'dataset'
        triples.append([term, actual_label, amount])

    return sorted_occurrences, sorted_upcoming, terms_labels, triples

In [55]:
popular, upcoming, entity_labels, entity_triples = popular_upcoming_entities(id_list)

In [58]:
for paper in id_list[:2]:
    _query_terms = {
        "query": {
            "bool": {
                "must": [{"match": {'paper_id': paper}}],
                "should": [{"match": {"annotator": 'method'}},
                           {"match": {"annotator": 'dataset'}}]
            }
        }
    }

    _query_terms = es.search(index="entities_tud", doc_type="entities", body=_query_terms)

    for hit in _query_terms['hits']['hits']:
        print(hit['_source']['lower'])

mapreduce
whereas
whenever
big data processing
big data analytics
scalability
stack
big data processing
bigbench
converted
matlab
whenever


In [59]:
hit

{'_id': 'TUD_98a8d01e-6259-4fed-8704-5af0615304d2138187',
 '_index': 'entities_tud',
 '_score': 46.910866,
 '_source': {'PMIdata': 0.6049066545457461,
  'PMImethod': 2.100680896118057,
  'annotator': 'undefined',
  'clean': 'whenever',
  'ds_sim_50': 0,
  'ds_sim_60': 0,
  'ds_sim_70': 0,
  'ds_sim_80': 0,
  'ds_sim_90': 0,
  'ds_similarity': 0.470866,
  'filteredWord': 'whenever',
  'inwordNet': 0,
  'journal': 'tudelft',
  'label': 'dataset',
  'lower': 'whenever',
  'mt_sim_50': 1,
  'mt_sim_60': 0,
  'mt_sim_70': 0,
  'mt_sim_80': 0,
  'mt_sim_90': 0,
  'mt_similarity': 0.474401,
  'no_punkt': 'whenever',
  'paper_id': 'TUD_98a8d01e-6259-4fed-8704-5af0615304d2',
  'title': 'Development of the Machine Learning-Based Approach to Access Occupancy Information Through Indoor CO2 Data: for Demand-Driven Building Heating Energy Management',
  'word': 'whenever',
  'year': '2017'},
 '_type': 'entities'}

# Data Preparation

In [4]:
import pymongo
import elasticsearch
import nltk
import string
import sys
import re
import os

import config as cfg

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
client = pymongo.MongoClient('localhost:' + str(cfg.mongoDB_Port))
publications_collection = client.TU_Delft_Library
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}],
                                 timeout=30, max_retries=10, retry_on_timeout=True)
es.cluster.health(wait_for_status='yellow', request_timeout=1)


def return_names(mongo_string_search, db):
    results = db.publications.find(mongo_string_search)
    list_of_docs = list()
    extracted = {
        "_id": "",
    }
    for i, r in enumerate(results):
        extracted['_id'] = r['_id']
        list_of_docs.append(extracted)
        extracted = {
            "_id": "",
        }
    return list_of_docs

In [5]:
extracted_publications = []
query = {}
extracted_publications.append(return_names(query, publications_collection))

In [6]:
extracted_publications

[[{'_id': 'TUD_862e5b58-b9d1-462a-90b0-6f94a054632e'},
  {'_id': 'TUD_960eb3cd-080d-4423-b9cb-f82417e9e13c'},
  {'_id': 'TUD_c35fa6e0-e632-4f17-bf84-0f1cc8f98c0c'},
  {'_id': 'TUD_f116361b-8ff7-4995-a8e5-e5354f2902b9'},
  {'_id': 'TUD_46081d76-17fc-41f6-bdc4-fbdc6d7bb939'},
  {'_id': 'TUD_ed609f8d-6cc5-489f-aa45-669b797f677a'},
  {'_id': 'TUD_f514548b-63c6-4015-a824-c86c9a63477c'},
  {'_id': 'TUD_beca2029-d274-4d14-9ea0-a59490019bc1'},
  {'_id': 'TUD_fc4aa0e6-1d20-41e9-a660-237a6873bb3c'},
  {'_id': 'TUD_84ff0f8e-ec0a-4b77-8fa9-9df7eaef6b60'},
  {'_id': 'TUD_9ced4cb2-9809-4b58-af25-34e458e8ea23'},
  {'_id': 'TUD_b0c2a859-4351-4b6a-b66b-a7bbfed76ecb'},
  {'_id': 'TUD_280fe4d8-d1de-4a46-ad99-be8129d85dab'},
  {'_id': 'TUD_78bf003b-e784-40ca-888c-48e0246b3883'},
  {'_id': 'TUD_ba867fef-5421-4692-b955-6c2c468653cc'},
  {'_id': 'TUD_b6ea3435-825a-4306-a4d4-4edee26f233c'},
  {'_id': 'TUD_8150156e-e633-4319-8685-ccac7b083434'},
  {'_id': 'TUD_c061a03a-84a5-4ac3-ba22-c5b315f08edd'},
  {'_id': 

# Training NER (module 1)

In [11]:
from m1_preprocessing import seed_data_extraction, term_sentence_expansion, training_data_generation, ner_training
from m1_postprocessing import extract_new_entities, filtering
import config as cfg
import gensim
import elasticsearch
import time
import re
doc2vec_model = gensim.models.Doc2Vec.load(cfg.ROOTPATH + '/embedding_models/doc2vec.model')
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [21]:
seeds = ['hierarchical agglomerative', 'selection algorithm', 'stochastic gradient descent', 'pearson correlation', 
         'semantic relevance', 'gpbased', 'pattern matching', 'clir', 'random forest', 'random indexing', 
         'basic load control method', 'linear regression', 'recursive function', 'latent dirichlet allocation', 
         'convolutional dnn', 'likelihood function', 'folding-in', 'restricted boltzmann machine', 'lstm', 
         'radial basis function network', 'bmecat', 'lib', 'fast fourier', 'adaptive filter', 'spectral clustering', 
         'dmp method', 'reinforcement learning', 'graph-based propagation', 'semantictyper', 'hierarchical clustering',
         'variational em', 'qald', 'fourier analysis', 'simple random algorithm', 'random search', 'lsh method', 
         'regular expression', 'rapid7', 'word embedding', 'autoencoder', 'bayesian nonparametric', 
         'variational bayesian inference', 'tsa algorithm', 'predictive modeling', 'query optimization', 'softmax', 
         'ridge regularization', 'tdcm', 'support vector machine', 'mcmc']

# seeds = ['buzzfeed', 'pslnl', 'dailymed', 'robust04', 'scovo', 'ask.com', 'cacm', 'stanford large network dataset', 'mediaeval', 'lexvo', 'spambase', 'shop.com', 'orkut', 'jnlpba', 'cyworld', 'citebase', 'blog06', 'worldcat', 'booking.com', 'semeval', 'imagenet', 'nasdaq', 'brightkite', 'movierating', 'webkb', 'ionosphere', 'moviepilot', 'duc2001', 'datahub', 'cifar', 'tdt', 'refseq', 'stack overflow', 'wikiwars', 'blogpulse', 'ws-353', 'gerbil', 'wikia', 'reddit', 'ldoce', 'kitti dataset', 'specweb', 'fedweb', 'wt2g', 'as3ap', 'friendfeed', 'new york times', 'chemid', 'imageclef', 'newegg']


context_words = ['method', 'algorithm', 'approach', 'evaluate']
sentence_expansion = True
training_cycles = 4 
model_name = 'method_tud'
filtering_pmi = True
filtering_st = False
filtering_ws = True
filtering_kbl = True
filtering_majority = True

cycle = 0

In [4]:
sentences, entities = seed_data_extraction.sentence_extraction(model_name, cycle, seeds)

Started initial training data extraction
Extracting sentences for 50 seed terms
..................................................

In [22]:
for entity in seeds:
        entity_name = re.sub(r'\([^)]*\)', '', entity)
        print('.', end='')
        query = {"query":
                    {"match":
                        {"content.chapter.sentpositive":
                            {"query": entity_name,
                             "operator": "and"
                             }
                         }
                     }
                 }

        res = es.search(index="twosent_tud", doc_type="twosentnorules",
                        body=query, size=1000)

        # clean up the sentences and if they don't contain the names of the test set then add them as
        # the training data
#         for doc in res['hits']['hits']:
        print(entity, len(res['hits']['hits']))

.hierarchical agglomerative 17
.selection algorithm 524
.stochastic gradient descent 29
.pearson correlation 93
.semantic relevance 38
.gpbased 0
.pattern matching 301
.clir 0
.random forest 98
.random indexing 2
.basic load control method 0
.linear regression 331
.recursive function 63
.latent dirichlet allocation 8
.convolutional dnn 2
.likelihood function 204
.folding-in 104
.restricted boltzmann machine 0
.lstm 5
.radial basis function network 3
.bmecat 0
.lib 54
.fast fourier 127
.adaptive filter 78
.spectral clustering 22
.dmp method 0
.reinforcement learning 14
.graph-based propagation 4
.semantictyper 0
.hierarchical clustering 81
.variational em 0
.qald 0
.fourier analysis 124
.simple random algorithm 18
.random search 160
.lsh method 0
.regular expression 176
.rapid7 2
.word embedding 28
.autoencoder 0
.bayesian nonparametric 6
.variational bayesian inference 0
.tsa algorithm 3
.predictive modeling 14
.query optimization 135
.softmax 3
.ridge regularization 20
.tdcm 0
.suppor

# Processing Entities

In [61]:
import gensim
import sys
import os
from nltk.tag.stanford import StanfordNERTagger
from elasticsearch import Elasticsearch
from pymongo import MongoClient
from nltk.corpus import wordnet
from nltk.corpus import stopwords
# from gensim.models.wrappers import FastText

from m1_postprocessing import process_extracted_entities
import config as cfg

# embedding_model = FastText.load_fasttext_format('/data/modelFT')
# embedding_model = gensim.models.Word2Vec.load('embedding_models/modelword2vecbigram.model')
embedding_model = gensim.models.Word2Vec.load('full_corpus_models/modelword2vecbigram.model')

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.cluster.health(wait_for_status='yellow')

client = MongoClient('localhost:' + str(cfg.mongoDB_Port))
db_mongo = client.TU_Delft_Library.entities
path_to_jar = cfg.STANFORD_NER_PATH
entity_names = []

In [62]:
def store_entity_in_mongo(db, _id, title, journal, year, word, in_wordnet, filtered_words, pmi_data, pmi_method,
                          ds_similarity, mt_similarity, ds_sim_50, ds_sim_60, ds_sim_70, ds_sim_80, ds_sim_90,
                          mt_sim_50, mt_sim_60, mt_sim_70, mt_sim_80, mt_sim_90):
    my_ner = {
        "paper_id": _id,
        "title": title,
        "journal": journal,
        "year": year,
        "word": word,
        "in_wordnet": in_wordnet,
        "filtered_word": filtered_word,
        "pmi_data": pmi_data,
        "pmi_method": pmi_method,
        "ds_similarity": ds_similarity,
        "mt_similarity": mt_similarity,
        "ds_sim_50": ds_sim_50,
        "ds_sim_60": ds_sim_60,
        "ds_sim_70": ds_sim_70,
        "ds_sim_80": ds_sim_80,
        "ds_sim_90": ds_sim_90,
        "mt_sim_50": mt_sim_50,
        "mt_sim_60": mt_sim_60,
        "mt_sim_70": mt_sim_70,
        "mt_sim_80": mt_sim_80,
        "mt_sim_90": mt_sim_90

    }

#     db.entities.update_one({'$and': [{'paper_id': my_ner['paper_id']}, {'word': word}]}, {'$set': my_ner}, upsert=True)

In [63]:
def get_entities(words, current_model):
    results = []
    facet_tag = current_model.upper()
    for i, (a, b) in enumerate(words):
        if b == facet_tag:
            temp = a
            if i > 1:
                j = i - 1
                if words[j][1] == facet_tag:
                    continue
            j = i + 1
            try:
                if words[j][1] == facet_tag:
                    temp = b
                    temp = words[j][0] + ' ' + b
                    z = j + 1
                    if words[j][1] == facet_tag:
                        temp = a + ' ' + words[j][0] + ' ' + words[z][0]
            except IndexError:
                continue
            results.append(temp)


    wordnet_filtered = []
    filtered_words = [word for word in set(results) if word not in stopwords.words('english')]

    filtered_words = [word for word in set(results) if word not in stopwords.words('english')]
    print(len(filtered_words), 'entities to process in this document')
    
    for word in set(filtered_words):
        in_wordnet = 1
        if not wordnet.synsets(word):
            wordnet_filtered.append(word)
            in_wordnet = 0

        filtered_word, pmi_data, pmi_method, ds_similarity, mt_similarity, ds_sim_50, \
        ds_sim_60, ds_sim_70, ds_sim_80, ds_sim_90, mt_sim_50, mt_sim_60, mt_sim_70, \
        mt_sim_80, mt_sim_90 = process_extracted_entities.filter_it(word, embedding_model)
        
        print(filtered_word, pmi_data, pmi_method, ds_similarity, mt_similarity)

In [6]:
publications = ['tudelft']
model_name = 'method_tud_full'

for publication in publications:
    query = {"query":
                {"match":
                    {"journal":
                        {"query": publication,
                         "operator": "and"
                         }
                     }
                 }
             }

    res = es.search(index="ir_tud", doc_type="publications",
                    body=query, size=10000)

    print(len(res['hits']['hits']), 'publications to process')

955 publications to process


In [9]:
print(len(res['hits']['hits']), 'publications to process')
prog = len(res['hits']['hits'])
path_to_model = 'crf_trained_files/' + model_name + '_TSE_model_0.ser.gz'
ner_tagger = StanfordNERTagger(path_to_model, path_to_jar)

for doc in res['hits']['hits'][:2]:
    text = doc["_source"]["content"]
    print(doc["_source"]["title"])
    labelled_words = ner_tagger.tag(text.split())
    get_entities(labelled_words, model_name)
    print(prog, 'more documents to go')
    prog = prog - 1
    print('')

955 publications to process
Modeling and Inversion of Scattered Surface waves


/usr/local/lib/python3.5/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


21 entities to process in this document
Fourier transformation given 0 0 0 0
Problem Results 2.7750146346224454 2.975827828907664 0 0
Fourier Transforms In 0 4.521796198012956 0 0
sequence expansion 0 3.6737992914580064 0 0
Fourier transform 1.661819561165165 3.383869658561152 0.019224574602318377 -0.005251271983596223
number model 2.7162790577988436 2.4984881377850257 0 0
conjugate gradient method 0 6.995727386345369 0 0
analytical expression 0 4.078189546537342 0 0
number 2.7174994785616624 2.4816776074273763 0.00345131345018591 -0.007272460138335303
Fourier transform 1.661819561165165 3.383869658561152 0.019224574602318377 -0.005251271983596223
number frequencies 0 1.5254074515653355 0 0
part 2.0034735801073804 2.0719213203249787 0.015343580136696854 0.006523893347861064
transform 2.5337531862654417 3.1997273592553404 -0.015198943795714124 0.029077524026399577
effect 2.3119554295121936 2.2518145284046653 -0.014363177827582797 -0.004376279637342002
number cells 2.382383548391881 2.47

In [67]:
model_name = 'dataset_100'
path_to_model = 'crf_trained_files/' + model_name + '_TSE_model_0.ser.gz'
ner_tagger = StanfordNERTagger(path_to_model, path_to_jar)
labelled_words = ner_tagger.tag(text.split())

/usr/local/lib/python3.5/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [19]:
from m1_postprocessing.filtering import normalized_entity_distance
# import gensim 

# embedding_model = gensim.models.Word2Vec.load('models/modelword2vecbigram.model')

ds_names = []
mt_names = []
# prnames = []

dataset_path = 'data/dataset_names2.txt'
with open(dataset_path, "r") as file:
    for row in file.readlines():
        ds_names.append(row.strip())

method_path = 'data/method_names2.txt'
with open(method_path, "r") as file:
    for row in file.readlines():
        mt_names.append(row.strip())
        
def is_int_or_float(s):
    """
    return 1 for int, 2 for float, -1 for not a number
    """
    try:
        float(s)
        return 1 if s.count('.') == 0 else 2
    except ValueError:
        return -1

In [24]:
ner_word = 'markov chains'
ner_word = ner_word.split()
ner_word

['markov', 'chains']

In [25]:
filter_by_wordnet = []
filtered_words = [word for word in ner_word if word not in stopwords.words('english')]

for word in filtered_words:
    isint = is_int_or_float(word)
    if isint != -1:
        filtered_words.remove(word)

for word in set(filtered_words):
    in_wordnet = 1
    inds = 0

    if not wordnet.synsets(word):
        in_wordnet = 0
        filter_by_wordnet.append(word)
        

In [26]:
filtered_words

['markov', 'chains']

In [27]:
filtered_words = ' '.join(filtered_words)
filtered_words = filtered_words.replace('(', '')
filtered_words = filtered_words.replace(')', '')
filtered_words = filtered_words.replace('[', '')
filtered_words = filtered_words.replace(']', '')
filtered_words = filtered_words.replace('{', '')
filtered_words = filtered_words.replace('}', '')
filtered_word = filtered_words.replace(',', '')
lower_filtered_word = filtered_word.lower()
filter_by_wordnet = ' '.join(filter_by_wordnet)

In [28]:
lower_filtered_word

'markov chains'

In [29]:
pmi_data = normalized_entity_distance(filtered_word, 'dataset')
pmi_method = normalized_entity_distance(filtered_word, 'method')
pmi_method, pmi_data

(2.908264545095029, 4.140410758085917)

In [30]:
ds_similarity = []
mt_similarity = []
lower_filtered_word = lower_filtered_word.replace(' ', '_')
lower_filtered_word

'markov_chains'

In [34]:
mt_similarity = []
x = 0
y = 0
for mt in mt_names:
    try:
        mt = mt.replace(' ', '_')
        similarity = embedding_model.wv.similarity(mt, lower_filtered_word)
        print(mt, similarity, '-'*50)
        mt_similarity.append(similarity)
        y = y + 1
    except KeyError:
#         print(mt, 'not in vectorspace')
        x = x + 1
print(y, x)
# try:
#     mt_similarity = float(sum(mt_similarity)) / len(mt_similarity)
# except ZeroDivisionError:
#     pass

print(mt_similarity)

simulated_annealing -0.07921258353650174 --------------------------------------------------
linear_regression 0.1696744497926577 --------------------------------------------------
genetic_programming -0.0703554376625785 --------------------------------------------------
qald 0.014135182261109129 --------------------------------------------------
downhill_simplex -0.0657312573852436 --------------------------------------------------
regular_expression -0.03982873864022178 --------------------------------------------------
selection_algorithm 0.19964432417909211 --------------------------------------------------
imrank -0.11654380197968744 --------------------------------------------------
adarank 0.028082375279707885 --------------------------------------------------
similarity_search 0.04953733805337633 --------------------------------------------------
game_theory -0.0413717585760423 --------------------------------------------------
mcmc -0.1346701443416963 --------------------------

In [37]:
query = {}
res = es.search(index="twosent_tud", doc_type="twosentnorules", body=query)
    total_a = res['hits']['total']

In [38]:
total_a

593162

# Editing Entities in Mongo

In [148]:
import nltk
from pymongo import MongoClient
from config import mongoDB_Port
import string

tr = str.maketrans("", "", string.punctuation)
client = MongoClient('localhost:' + str(mongoDB_Port))
db = client.TU_Delft_Library
pub = db.publications
es = Elasticsearch(
    [{'host': 'localhost', 'port': 9200}], timeout=30, max_retries=10, retry_on_timeout=True
)
es.cluster.health(wait_for_status='yellow', request_timeout=1)
allentities = db.entities.find()
words = []
x = 0
for rr in allentities:
    word = rr['word'].lower()
    no_punkt = word.translate(tr)
    clean = ''.join([i for i in no_punkt if not i.isdigit()])
    lower = rr['word'].lower()
    
    db.entities.update_one({'_id' : rr['_id']}, 
                           {"$set" : {'clean': clean, 'no_punkt': no_punkt,
                                      'word_lower' : lower}}, upsert = False)
    x = x + 1
    if x % 10000 == 0:
        print(x)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [149]:
tud_db = client.TU_Delft_Library
pub_db = client.pub
tud_entities = tud_db.entities.find()
pub_entities = pub_db.entities.find()

In [66]:
ann = {}
for rr in pub_entities:
    if rr['Annotator'] != 'undefined':
        ann[rr['word_lower']] = rr['Annotator']

In [70]:
tud_entities = tud_db.entities.find()
for rr in tud_entities:
    try:
        new = ann[rr['word_lower']]
        db.entities.update_one({'_id' : rr['_id']}, {"$set" : {'Annotator' : new}}, upsert = False)
    except KeyError:
        pass

In [150]:
tud_entities = tud_db.entities.find()
for rr in tud_entities:
    db.entities.update_one({'_id' : rr['_id']}, {"$set" : {'experiment' : '1cycle_tud_word2vec'}}, upsert = False)